## Elliptic Curve Cryptography
Within this project we are implementing the Elliptic Curve Cryptography (ECC) algorithm, and exploring two different attack methods. If you have questions on the functionality of the ECC algorithm please refer to the pdf file also included within this github repository. 

Beneath we just define a few functions that we will use in the implementation of the ECC algorithm. All of these are derived from Number Theory Algorithms or Theorems.

In [14]:
#breaks down a composite number into prime factors 
def factorize(n):
    factors = []
    p = 2
    while True:
        while n % p == 0 and n > 0:  
            factors.append(p)
            n = n / p
        p += 1  
        if p > n / p:
            break
    if n > 1:
        factors.append(n)
    return factors

#checks if a number is prime, for larger numbers witness checks are better but this is more efficent in the small case 
def isPrime(a):
    a = int(a)
    if a%2 == 0:
        return False
    for i in range (3, a, 2):
        if a % i == 0:
            return False
    return True

#calculates the legendre symbol of a mod p 
def legendre_symbol(a, p):
    #checks p is prime 
    if not isPrime(p):
        raise ValueError(f"{p} is not a prime number")
    #mods it if we can to make it smaller
    if a >= p or a < 0:
        return legendre_symbol(a % p, p)
    #if its 1 or 0 its always 1
    elif a == 0 or a == 1:
        return a
    #The 2 case 
    elif a == 2:
        if p % 8 == 1 or p % 8 == 7:
            return 1
        else:
            return -1
    #if a is p-1 then a is congruent to -1 mod p and we have a special case 
    elif a == p - 1:
        if p % 4 == 1:
            return 1
        else:
            return -1

    #if a is not prime we break it down into its prime factors and calculate for those 
    elif not isPrime(a):
        factors = factorize(a)
        product = 1
        for pi in factors:
            product *= legendre_symbol(pi, p)
        return product
    else:
        #flip the symbol if a or p is 3 mod 4
        if a%4==1 or p%4==1:
            return legendre_symbol(p, a)
        else:
            return (-1) * legendre_symbol(p, a)

#this algorithm calculates the square root of a mod p
def tonelli_shanks(a, p):
    if a == 0:
        return 0

    q, s = p - 1, 0
    while q % 2 == 0:
        q //= 2
        s += 1

    z = 2
    while pow(z, (p - 1) // 2, p) == 1:
        z += 1

    m = s
    c = pow(z, q, p)
    t = pow(a, q, p)
    r = pow(a, (q + 1) // 2, p)

    while t != 0 and t != 1:
        t2i = t
        i = 0
        for i in range(1, m):
            t2i = pow(t2i, 2, p)
            if t2i == 1:
                break

        b = pow(c, 2 ** (m - i - 1), p)
        m = i
        c = pow(b, 2, p)
        t = (t * c) % p
        r = (r * b) % p

    return r if t == 1 else None

#just  a simple gcd function
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

#calculates the linear combination of a and b that equal their gcd 
def extended_gcd(a, b):
    if b == 0:
        return a, 1, 0
    gcd, x1, y1 = extended_gcd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return gcd, x, y

#This uses the EEA to find the modular inverse of a number
def mod_inverse(a, m):
    gcd, x, _ = extended_gcd(a, m)
    if gcd != 1:
        raise ValueError(f"Inverse of {a} does not exist modulo {m}")
    return x % m

legendre_symbol(5, 19)

1

Now we are going to create a object called EllipticCurve which will contain most of important information about the curve and functions that will be preformed on the curve itself. 

In [15]:
class EllipticCurve:
    #defines an elliptic curve of the form y^2 = x^3 + ax + b (mod p) with None as the point at infinity
    def __init__(self, a, b, p):
        self.a = a
        self.b = b
        self.p = p

    def is_on_curve(self, x, y):
        #checks if both sides equal each other AKA if the point is on the curve
        return ((y**2) % self.p) == ((x**3 + self.a * x + self.b) % self.p)
    
    #returns just the x side of the equation
    def x_side(self, x):
        return (x**3 + self.a * x + self.b) % self.p

    def point_addition(self, P, Q):
        #With None representing the point at infinity
        if P is None: return Q
        if Q is None: return P
    
        x1, y1 = P
        x2, y2 = Q

        #this case returns the point at infinity 
        if x1 == x2 and y1 == -y2 % self.p:
            return None

        #this calculates the slope of the line between the two points
        if P == Q:
            m = (3 * x1**2 + self.a) * pow(2 * y1, -1, self.p) % self.p
        else:
            m = (y2 - y1) * pow(x2 - x1, -1, self.p) % self.p
        
        #with the slope we calculate the new point
        x3 = (m**2 - x1 - x2) % self.p
        y3 = (m * (x1 - x3) - y1) % self.p

        return (x3, y3)
    
    
    #this function just repeated adds the point to itself k times
    def scalar_multiplication(self, k, P):
        R = None
        Q = P

        while k > 0:
            if k % 2 == 1:
                R = self.point_addition(R, Q)
            Q = self.point_addition(Q, Q)
            k //= 2

        return R
    
    #allows us to print the curve easier 
    def __str__(self):
        return f"Our Elliptic Curve is x^3 + {self.a}x + {self.b} (mod {self.p})"
    
    #checks if the curve is singular by checking if the discriminant is not 0
    def check_valid_discrimant(self):
        return 4 * self.a**3 + 27 * self.b**2 != 0
    
    #calculates the number of points on the curve using Legendre Symbols 
    def num_of_points(self):
        count = 1
        for i in range(self.p):
            if legendre_symbol(self.x_side(i), self.p) == 1:
                count += 2
        return count

This part shows the actual encrpytion within action. Within this the only public things are the curve itself and the Bob and Alices public keys. The shared secret they create with each other is not publicly known.

In [16]:
import random
import sympy

size_of_prime = 500

#generates a random elliptic curve
curve = None
while curve is None or curve.check_valid_discrimant() == False:
    p = sympy.randprime(10, size_of_prime)
    a = random.randint(0, p)
    b = random.randint(0, p)
    curve = EllipticCurve(a, b, p)
print(f"The curve is {curve}")

#generates a random point on the curve using Legendre Symbols and Tonneli Shanks
def generate_random_point(curve):
    while True:
        x = random.randint(0, curve.p)
        #check if x is a square mod p
        if legendre_symbol(curve.x_side(x), curve.p) != 1:
            continue
        #find y value for x 
        y = tonelli_shanks(curve.x_side(x), curve.p)
        if curve.is_on_curve(x, y):
            return (x, y)
        
#random point on the curve that will be our generator point
G = generate_random_point(curve)
print(f"Our generator point is {G}")
print("\n")

#Alice and Bob generate their private keys
alice_priv = random.randint(1, curve.p)
bob_priv = random.randint(1, curve.p)
print(f"Our Alice Private key {alice_priv}")
print(f"Our Bob Private key {bob_priv}")
print("\n")

#they then generate their public keys by multiplying the generator point by their private key
alice_pub = curve.scalar_multiplication(alice_priv, G)
bob_pub = curve.scalar_multiplication(bob_priv, G)
print(f"Alice Public Key {alice_pub}")
print(f"Bob Public Key {bob_pub}")
print("\n")

#shared secrets are made by multiplying the other persons public key by your private key
alice_shared_secret = curve.scalar_multiplication(alice_priv, bob_pub)
bob_shared_secret = curve.scalar_multiplication(bob_priv, alice_pub)

print(f"Alice Shared Secret {alice_shared_secret}")
print(f"Bob Shared Secret {bob_shared_secret}")




The curve is Our Elliptic Curve is x^3 + 168x + 67 (mod 223)
Our generator point is (51, 166)


Our Alice Private key 201
Our Bob Private key 67


Alice Public Key (115, 222)
Bob Public Key (3, 44)


Alice Shared Secret (14, 82)
Bob Shared Secret (14, 82)


Now this part shows different cryptographic attacks that can be performed on the ECC algorithm. The first attack is the brute force attack, which simply tries to solve the discrete logarithm problem by trying higher and higher values until the correct one is found. The second attack is the Pollard's rho algorithm, which uses random walks to find the discrete logarithm. Phollard Rho is more efficent at larger scales and can actually be distributed on multiple machines (each one is doing their own random walk). 

In [17]:
from tqdm import tqdm
import time

#This is a simple brute force attack iterating and checking 
def brute_force(curve, G, alice_pub):
    found_priv = None
    # Iterate through all possible private keys
    for priv_guess in tqdm(range(1, curve.p-1), desc="Percentage of all keys checked"):
        guess_pub = curve.scalar_multiplication(priv_guess, G)

        #if one works we break out of the loop
        if guess_pub == alice_pub:
            found_priv = priv_guess
            break
    
    #if we don't find the private key have this print statement inform us what happened 
    if found_priv is None:
        print("Failed to find Alice's private key.")
    return found_priv

import random
def pollards_rho_ecdlp(curve, G, pub_key):
    #This is the parition function that allows for a psuedo-random walk
    def partition(P):
        if P is None:
            return 2
        return P[0] % 3
    
    class Point:
        def __init__(self, a, b):
            self.a = a
            self.b = b
            self.loc = curve.point_addition(curve.scalar_multiplication(a, G), curve.scalar_multiplication(b, pub_key))

        def __eq__(self, other):
            return self.x == other.x and self.y == other.y

        def __repr__(self):
            return f"{self.loc} = {self.a}G + {self.b}P"
        
        def deepcopy(self):
            return Point(self.a, self.b)
        
        def step(self):
            partition_idx = partition(self.loc)
            if partition_idx == 0:
                self.a = (self.a + 1) % num_points
                self.loc = curve.point_addition(self.loc, G)
            elif partition_idx == 1:
                self.b = (self.b + 1) % num_points
                self.loc = curve.point_addition(self.loc, pub_key)
            else:
                self.a = (2 * self.a) % num_points
                self.b = (2 * self.b) % num_points
                self.loc = curve.scalar_multiplication(2, self.loc)
        
        def __eq__(self, other):
            return self.loc == other.loc

    #computes the number of points on the curve 
    num_points = curve.num_of_points()

    #starting point for the tortoise and the hare
    Tort = Point(random.randint(1, num_points - 1), random.randint(1, num_points - 1))
    Hare = Tort.deepcopy()


    while True:
        Tort.step()

        Hare.step()
        Hare.step()

        #detects a collision 
        if Hare == Tort:
            numerator = (Tort.a - Hare.a) % num_points
            denominator = (Hare.b - Tort.b) % num_points

            # Attempts to solve for the private key
            if gcd(denominator, num_points) == 1:
                inverse_denominator = mod_inverse(denominator, num_points)
                private_key = (numerator * inverse_denominator) % num_points
                if curve.scalar_multiplication(private_key, G) == pub_key:
                    return private_key
            else:
                # if it fails it tries again 
                Tort = Point(random.randint(1, num_points - 1), random.randint(1, num_points - 1))
                Hare = Tort.deepcopy()  



print("Brute force attack")
start = time.time()
stolen_key=brute_force(curve, G, alice_pub)
print(f"Stolen key: {stolen_key}")
stolen_shared_secret = curve.scalar_multiplication(stolen_key, bob_pub)
print(f"Stolen shared secret: {stolen_shared_secret}")
print(f"Actual shared secret:  {alice_shared_secret}")
end = time.time()
print(f"Time taken: {end-start}")
print("\n")
print("\n")


print("Pollards Rho attack")
start = time.time()
stolen_key=pollards_rho_ecdlp(curve, G, alice_pub)
print(f"Stolen key: {stolen_key}")
print(f"Alice private key: {alice_priv}")
stolen_shared_secret = curve.scalar_multiplication(stolen_key, bob_pub)
print(f"Stolen shared secret: {stolen_shared_secret}")
print(f"Actual shared secret:  {alice_shared_secret}")
end = time.time()
print(f"Time taken: {end-start}")

Brute force attack


Percentage of all keys checked:   9%|▉         | 20/221 [00:00<00:00, 143886.93it/s]

Stolen key: 21
Stolen shared secret: (14, 82)
Actual shared secret:  (14, 82)
Time taken: 0.029188871383666992




Pollards Rho attack
Stolen key: 81
Alice private key: 201
Stolen shared secret: (14, 82)
Actual shared secret:  (14, 82)
Time taken: 0.012306928634643555
